# We need to do augmentations too, as done in nate gruver paper.

In [1]:
import os
from tqdm import tqdm

mp_path = "/scratch/civil/phd/cez198233/vaibhav_nlp/corpus/materials_project_cifs"
ruff_path = "/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/amcsd"
gnome_path = "/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs"

In [2]:
mp_files = os.listdir(mp_path)

In [3]:
important_keywords = ['_cell_length_a', '_cell_length_b', '_cell_length_c', '_cell_angle_alpha', '_cell_angle_beta', '_cell_angle_gamma', '_chemical_formula_sum', '_cell_formula_units_Z', '_symmetry_space_group_name_H-M', '_chemical_formula_structural']
redundant_keywords = ['_symmetry_Int_Tables_number', '_chemical_formula_structural', '_cell_volume', '#', 'data_']

In [4]:
# empty = []
# for idx in tqdm(range(len(mp_files))):
#     try:
#         cif = open(os.path.join(mp_path, mp_files[idx], mp_files[idx] + '.cif'), 'r').readlines()
#         cnt_loop = 0
#         for i in cif:
#             if 'loop' in i:
#                 cnt_loop += 1
#         if cnt_loop != 2:
#             print(cif)
#             break
#     except:
#         empty.append(os.path.join(mp_path, mp_files[idx]))

In [11]:
cifs = {}

for folder in tqdm(mp_files):
    try:
        final_path = os.path.join(mp_path, folder+f"/{folder}.cif")
        cif = open(final_path, 'r').readlines()
    except Exception as e:
        continue
    _cif = {}
    num_data_lines = 0
    which_loop = None
    for line in cif:
        if which_loop:
            _cif[which_loop].append(line)
            continue
        if line.startswith('#'):
            continue
        if line.startswith('data'):
            if num_data_lines == 1:
                print(folder, "has more than 1 data lines")
            num_data_lines += 1
            continue
        if line.startswith('loop_'):
            assert not ('l1' in _cif and 'l2' in _cif)
            if 'l1' in _cif:
                which_loop = 'l2'
            else:
                which_loop = 'l1'
            _cif[which_loop] = []
            continue
        if len(line.split()) == 1:
            print("Oh this is extra.")
            print(folder, line, _cif.keys())
            continue
        line_features = line.split()
        if line_features[0] in redundant_keywords:
            continue
        if line_features[0] in important_keywords:
            _cif[line_features[0]] = ' '.join(line_features[1:])
            continue
        print("new_kw", folder, line)
    cifs[folder] = _cif

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 145919/145919 [22:22<00:00, 108.69it/s]


In [24]:
l1_start =  [' _symmetry_equiv_pos_site_id\n',
   ' _symmetry_equiv_pos_as_xyz\n',
   "  1  'x, y, z'\n",
   'loop_\n',
   ' _atom_site_type_symbol\n',
   ' _atom_site_label\n',
   ' _atom_site_symmetry_multiplicity\n',
   ' _atom_site_fract_x\n',
   ' _atom_site_fract_y\n',
   ' _atom_site_fract_z\n',
   ' _atom_site_occupancy\n',]
for i in tqdm(cifs.values()):
    for j in range(len(l1_start)):
        assert i['l1'][j] == l1_start[j]
    for j in range(len(l1_start), len(i['l1'])):
        b = i['l1'][j].split()
        assert b[2] == '1'
        assert b[-1] == '1'

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 143164/143164 [00:02<00:00, 49781.17it/s]


In [31]:
for cif in cifs:
    file = cifs[cif]
    file['unit_cell_structure'] = [i.split() for i in file['l1'][len(l1_start):]]
    file['unit_cell_structure'] = [[i[0], round(float(i[3]),1), round(float(i[4]),1), round(float(i[5]),1)] for i in file['unit_cell_structure']]
    file['_cell_length_a'] = round(float(file['_cell_length_a']), 1)
    file['_cell_length_b'] = round(float(file['_cell_length_b']), 1)
    file['_cell_length_c'] = round(float(file['_cell_length_c']), 1)
    file['_cell_angle_alpha'] = round(float(file['_cell_angle_alpha']), 1)
    file['_cell_angle_beta'] = round(float(file['_cell_angle_beta']), 1)
    file['_cell_angle_gamma'] = round(float(file['_cell_angle_gamma']), 1)
    file['_cell_formula_units_Z'] = int(file['_cell_formula_units_Z'])

In [33]:
for cif in cifs:
    cifs[cif].pop('l1')

In [35]:
a = set()
for cif in cifs:
    a.add(cifs[cif]['_symmetry_space_group_name_H-M'].split(' ')[1])

In [260]:
import pandas as pd
ms = []
ms2 = []

In [261]:
_pth = '/home/cse/btech/cs1200448/crystal-text-llm/data/with_tags'
files = os.listdir(_pth)[1:]
for file in files:
    print(file)
    ms.append(pd.read_csv(os.path.join(_pth, file)).to_dict())
    ms2.append(pd.read_csv(os.path.join(_pth, file)))

train_3.csv
test.csv
train_2.csv
train_6.csv
val.csv
train_7.csv
train_4.csv
train_0.csv
train_9.csv
train_5.csv
train_1.csv
train_8.csv


In [262]:
_pth = '/home/cse/btech/cs1200448/crystal-text-llm/data/basic'
files = os.listdir(_pth)[1:]
for file in files:
    print(file)
    ms.append(pd.read_csv(os.path.join(_pth, file)).to_dict())
    ms2.append(pd.read_csv(os.path.join(_pth, file)))

test.csv
val.csv
train.csv


In [123]:
materials = {}
for df in ms:
    for i in range(len(df['material_id'])):
        if df['material_id'][i] in materials:
            continue
        else:
            materials[df['material_id'][i]] = [(df['formation_energy_per_atom'][i], df['band_gap'][i], df['pretty_formula'][i], df['e_above_hull'][i], df['spacegroup.number'][i]), df['cif'][i]]

In [122]:
len(cifs)

143164

In [145]:
import numpy as np
props = ['formation_energy_per_atom', 'band_gap', 'pretty_formula', 'e_above_hull', 'space_group_number']
for m in materials:
    if m in cifs:
        for _, prop in enumerate(materials[m][0]):
            cifs[m][props[_]] = prop
    else:
        _cif = {}
        num_data_lines = 0
        which_loop = None
        for line in materials[m][1].strip().split('\n'):
            if which_loop:
                _cif[which_loop].append(line)
                continue
            if line.startswith('#'):
                continue
            if line.startswith('data'):
                if num_data_lines == 1:
                    print(folder, "has more than 1 data lines")
                num_data_lines += 1
                continue
            if line.startswith('loop_'):
                assert not ('l1' in _cif and 'l2' in _cif)
                if 'l1' in _cif:
                    which_loop = 'l2'
                else:
                    which_loop = 'l1'
                _cif[which_loop] = []
                continue
            if len(line.split()) == 1:
                print("Oh this is extra.")
                print(folder, line, _cif.keys())
                continue
            line_features = line.split()
            if line_features[0] in redundant_keywords:
                continue
            if line_features[0] in important_keywords:
                _cif[line_features[0]] = ' '.join(line_features[1:])
                continue
            print("new_kw", folder, line)
        _cif['unit_cell_structure'] = [i.split() for i in _cif['l1'][len(l1_start):]]
        print(_cif)
        _cif['unit_cell_structure'] = [[i[0], round(float(i[3]),1), round(float(i[4]),1), round(float(i[5]),1)] for i in _cif['unit_cell_structure']]
        _cif['_cell_length_a'] = round(float(_cif['_cell_length_a']), 1)
        _cif['_cell_length_b'] = round(float(_cif['_cell_length_b']), 1)
        _cif['_cell_length_c'] = round(float(_cif['_cell_length_c']), 1)
        _cif['_cell_angle_alpha'] = round(float(_cif['_cell_angle_alpha']), 1)
        _cif['_cell_angle_beta'] = round(float(_cif['_cell_angle_beta']), 1)
        _cif['_cell_angle_gamma'] = round(float(_cif['_cell_angle_gamma']), 1)
        _cif['_cell_formula_units_Z'] = int(_cif['_cell_formula_units_Z'])
        cifs[m] = _cif
        for _, prop in enumerate(materials[m][0]):
            cifs[m][props[_]] = prop
        

In [149]:
for cif in cifs:
    cifs[cif].pop('_symmetry_space_group_name_H-M')

In [426]:
# import pickle as pkl
# with open('/scratch/cse/btech/cs1200448/MatLlama/processed_cifs/mp_cifs.pkl', 'wb') as f:
#     pkl.dump(cifs, f)

# Gnome

In [231]:
import os
from tqdm import tqdm

gnome_path = "/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs"
gnome_files = os.listdir(gnome_path)
gnome_cifs = [os.listdir(os.path.join(gnome_path, i)) for i in gnome_files]
gnome_cif_paths = []
for _,i in enumerate(gnome_cifs):
    if gnome_files[_] == '250000_id':
        continue
    gnome_cif_paths += [os.path.join(gnome_path, gnome_files[_], x) for x in i]

In [233]:
exc = os.listdir('/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/250000_id/250000_id')
for i in exc:
    gnome_cif_paths.append(os.path.join('/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/250000_id/250000_id', i))

In [207]:
for i in gnome_cif_paths:
    if not i.endswith('.CIF'):
        print(i)

/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/310000_id/310000_id
/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/190000_id/stable_materials_summary.csv
/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/250000_id/250000_id
/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/280000_id/Book2.xlsx


In [235]:
important_keywords = ['_cell_length_a', '_cell_length_b', '_cell_length_c', '_cell_angle_alpha', '_cell_angle_beta', '_cell_angle_gamma', '_chemical_formula_sum', '_cell_formula_units_Z', '_symmetry_space_group_name_H-M', '_chemical_formula_structural']
redundant_keywords = ['_symmetry_Int_Tables_number', '_chemical_formula_structural', '_cell_volume', '#', 'data_']

In [236]:
for idx in tqdm(range(len(gnome_cif_paths))):
    try:
        cif = open(gnome_cif_paths[idx], 'r').readlines()
        cnt_loop = 0
        for i in cif:
            if 'loop' in i:
                cnt_loop += 1
        if cnt_loop != 2:
            print("doesnot have 2 loops", gnome_cif_paths[idx])
    except:
        print(gnome_cif_paths[idx])

  5%|█████████▌                                                                                                                                                                          | 20365/384941 [04:23<1:30:55, 66.83it/s]

/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/310000_id/310000_id


 68%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                       | 263089/384941 [1:01:47<15:47:38,  2.14it/s]

doesnot have 2 loops /scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/190000_id/stable_materials_summary.csv


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                  | 345627/384941 [1:18:45<11:25, 57.36it/s]

/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/280000_id/Book2.xlsx


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384941/384941 [1:24:54<00:00, 75.57it/s]


In [237]:
cifs = {}

for folder in tqdm(gnome_cif_paths):
    if not folder.endswith('CIF'):
        continue
    cif = open(folder, 'r').readlines()
    _cif = {}
    num_data_lines = 0
    which_loop = None
    for line in cif:
        if which_loop:
            _cif[which_loop].append(line)
            continue
        if line.startswith('#'):
            continue
        if line.startswith('data'):
            if num_data_lines == 1:
                print(folder, "has more than 1 data lines")
            num_data_lines += 1
            continue
        if line.startswith('loop_'):
            assert not ('l1' in _cif and 'l2' in _cif)
            if 'l1' in _cif:
                which_loop = 'l2'
            else:
                which_loop = 'l1'
            _cif[which_loop] = []
            continue
        if len(line.split()) == 1:
            print("Oh this is extra.")
            print(folder, line, _cif.keys())
            continue
        line_features = line.split()
        if line_features[0] in redundant_keywords:
            continue
        if line_features[0] in important_keywords:
            _cif[line_features[0]] = ' '.join(line_features[1:])
            continue
        print("new_kw", folder, line)
    cifs[folder.split('/')[-1].split('.cif')[0]] = _cif

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384941/384941 [1:33:14<00:00, 68.81it/s]


In [238]:
l1_start =  [' _symmetry_equiv_pos_site_id\n',
   ' _symmetry_equiv_pos_as_xyz\n',
   "  1  'x, y, z'\n",
   'loop_\n',
   ' _atom_site_type_symbol\n',
   ' _atom_site_label\n',
   ' _atom_site_symmetry_multiplicity\n',
   ' _atom_site_fract_x\n',
   ' _atom_site_fract_y\n',
   ' _atom_site_fract_z\n',
   ' _atom_site_occupancy\n',]
for i in tqdm(cifs.values()):
    for j in range(len(l1_start)):
        assert i['l1'][j] == l1_start[j]
    for j in range(len(l1_start), len(i['l1'])):
        b = i['l1'][j].split()
        assert b[2] == '1'
        assert b[-1] == '1'

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384938/384938 [00:06<00:00, 57330.55it/s]


In [239]:
for cif in cifs:
    file = cifs[cif]
    file['unit_cell_structure'] = [i.split() for i in file['l1'][len(l1_start):]]
    file['unit_cell_structure'] = [[i[0], round(float(i[3]),1), round(float(i[4]),1), round(float(i[5]),1)] for i in file['unit_cell_structure']]
    file['_cell_length_a'] = round(float(file['_cell_length_a']), 1)
    file['_cell_length_b'] = round(float(file['_cell_length_b']), 1)
    file['_cell_length_c'] = round(float(file['_cell_length_c']), 1)
    file['_cell_angle_alpha'] = round(float(file['_cell_angle_alpha']), 1)
    file['_cell_angle_beta'] = round(float(file['_cell_angle_beta']), 1)
    file['_cell_angle_gamma'] = round(float(file['_cell_angle_gamma']), 1)
    file['_cell_formula_units_Z'] = int(file['_cell_formula_units_Z'])

In [240]:
for cif in cifs:
    cifs[cif].pop('l1')

In [241]:
a = set()
for cif in cifs:
    a.add(cifs[cif]['_symmetry_space_group_name_H-M'].split(' ')[1])

In [243]:
for cif in cifs:
    cifs[cif].pop('_symmetry_space_group_name_H-M')

In [245]:
import pandas as pd
df = pd.read_csv('/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs/190000_id/stable_materials_summary.csv')

In [250]:
cnt = 0
for i in df['MaterialId']:
    if i+'.CIF' in cifs:
       cnt += 1
cnt

383680

In [334]:
df = df[['Composition', 'MaterialId', 'Reduced Formula', 'Density', 'Space Group Number', 'Crystal System', 'Corrected Energy', 'Bandgap', 'Formation Energy Per Atom', 'Decomposition Energy Per Atom']]
df.index = df.MaterialId
df

,Composition,MaterialId,Reduced Formula,Density,Space Group Number,Crystal System,Corrected Energy,Bandgap,Formation Energy Per Atom,Decomposition Energy Per Atom
MaterialId,,,,,,,,,,
000006a8c4,Cs1S6Zr3,000006a8c4,Cs(ZrS2)3,3.2834,12,monoclinic,-70.4155,0.0023,-1.9058,-0.1625
00011f62be,Nd7Os1Pr3Si5,00011f62be,Pr3Nd7Si5Os,6.8040,1,triclinic,-94.0804,0.0022,-0.5004,-0.0137
00012feb99,Ce2La3Pt16Tm15,00012feb99,La3Ce2Tm15Pt16,12.7292,1,triclinic,-237.6016,NaN,-1.2960,-0.1195
00014cac13,Mn1Ni1Os2Sb12Yb1,00014cac13,YbMnNi(Sb6Os)2,8.6776,2,triclinic,-93.6931,0.0011,-0.3064,-0.0837
1759321,Er8Ge4Si12Zr12,1759321,Er2Zr3Si3Ge,6.9851,1,triclinic,-254.0953,inf,-0.8718,-0.0186
...,...,...,...,...,...,...,...,...,...,...
ffff0b1194,Ga3Lu2Si1Tb4,ffff0b1194,Tb4Lu2Ga3Si,8.2687,38,orthorhombic,-48.0143,0.0009,-0.5925,-0.0143
ffff3e9e89,Au1Cd3Sm3Sn2,ffff3e9e89,Sm3Cd3Sn2Au,8.8579,189,hexagonal,-33.6678,0.0198,-0.6056,-0.0218
ffff611aad,Ba1H30Th7,ffff611aad,BaTh7H30,7.5986,146,trigonal,-178.4183,2.5688,-0.6005,-0.0252


In [407]:
props = ['formation_energy_per_atom', 'band_gap', 'pretty_formula', 'e_above_hull', 'space_group_number']
cnt = 0
mats = set(df.MaterialId)
for cif in tqdm(cifs):
    mid = cif.split('.CIF')[0]
    if mid in mats:
        m = df[df.index.isin([mid])].iloc[0]
        cifs[cif]['pretty_formula'] = m['Reduced Formula']
        cifs[cif]['formation_energy_per_atom'] = m['Formation Energy Per Atom']
        cifs[cif]['density'] = m['Density']
        cifs[cif]['space_group_number'] = m['Space Group Number']
        cifs[cif]['crystal_system'] = m['Crystal System']
        cifs[cif]['decomposition_energy_per_atom'] = m['Decomposition Energy Per Atom']
        if not np.isnan(m['Bandgap']):
            cifs[cif]['bandgap'] = ['Bandgap']

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384938/384938 [2:41:17<00:00, 39.78it/s]


In [436]:
# import pickle as pkl
# with open('/scratch/cse/btech/cs1200448/MatLlama/processed_cifs/gnome_cifs.pkl', 'wb') as f:
#     pkl.dump(cifs, f)

In [480]:
keys

{'_cell_angle_alpha',
 '_cell_angle_beta',
 '_cell_angle_gamma',
 '_cell_formula_units_Z',
 '_cell_length_a',
 '_cell_length_b',
 '_cell_length_c',
 '_chemical_formula_sum',
 'band_gap',
 'crystal_system',
 'decomposition_energy_per_atom',
 'density',
 'e_above_hull',
 'formation_energy_per_atom',
 'pretty_formula',
 'space_group_number',
 'unit_cell_structure'}

In [481]:
len(g) + len(cifs)

536483

# Flattening the folders

In [486]:
out_gnome = '/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs_all'
out_mp = '/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/mpcifs_all'
in_gnome = '/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/gnomecifs'
in_mp = '/scratch/civil/phd/cez198233/vaibhav_nlp/corpus/materials_project_cifs'
import shutil

In [495]:
err = []
for i,j,k in tqdm(os.walk(in_mp)):
    for _ in k:
        if not _.endswith('.cif'):
            continue
        full_pth = os.path.join(i,_)
        try:
            shutil.copy(full_pth, os.path.join(out_mp, _))
        except Exception as e:
            print(full_pth,i,j,k)
            err.append(full_pth)

145920it [35:03, 69.36it/s] 


In [497]:
ms = []
_pth = '/home/cse/btech/cs1200448/crystal-text-llm/data/with_tags'
files = os.listdir(_pth)[1:]
for file in files:
    print(file)
    ms.append(pd.read_csv(os.path.join(_pth, file)))
_pth = '/home/cse/btech/cs1200448/crystal-text-llm/data/basic'
files = os.listdir(_pth)[1:]
for file in files:
    print(file)
    ms.append(pd.read_csv(os.path.join(_pth, file)))

train_3.csv
test.csv
train_2.csv
train_6.csv
val.csv
train_7.csv
train_4.csv
train_0.csv
train_9.csv
train_5.csv
train_1.csv
train_8.csv
test.csv
val.csv
train.csv


In [506]:
mids = os.listdir(out_mp)
mids = set(mids)

In [502]:
ms[0]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup.number
0,38283,427,427,mp-1049239,-2.749529,0.0000,Ti2ZnP2O9,0.133611,"['O', 'P', 'Ti', 'Zn']",# generated using pymatgen\ndata_Ti2ZnP2O9\n_s...,62
1,38284,489,489,mp-1211593,-3.622006,4.8612,LiEr6B3O14,0.017893,"['B', 'Er', 'Li', 'O']",# generated using pymatgen\ndata_LiEr6B3O14\n_...,14
2,38285,95,95,mp-20740,-0.488500,0.1037,Tl4Te3Pb,0.000000,"['Pb', 'Te', 'Tl']",# generated using pymatgen\ndata_Tl4Te3Pb\n_sy...,140
3,38286,107,107,mp-756941,-1.417111,0.0000,LiFeO3,0.148444,"['Fe', 'Li', 'O']",# generated using pymatgen\ndata_LiFeO3\n_symm...,52
4,38287,118,118,mp-1173345,-3.142472,2.1708,PaInO4,0.000000,"['In', 'O', 'Pa']",# generated using pymatgen\ndata_PaInO4\n_symm...,141
...,...,...,...,...,...,...,...,...,...,...,...
12756,51039,464,464,mp-555791,-3.227390,5.7384,SiO2,0.040642,"['O', 'Si']",# generated using pymatgen\ndata_SiO2\n_symmet...,229
12757,51040,473,473,mp-1191741,-2.528763,0.0000,CaSiO4,0.345235,"['Ca', 'O', 'Si']",# generated using pymatgen\ndata_CaSiO4\n_symm...,43
12758,51041,320,320,mp-756560,-3.192929,2.2641,Li2Ti3O7,0.049609,"['Li', 'O', 'Ti']",# generated using pymatgen\ndata_Li2Ti3O7\n_sy...,15
12759,51042,355,355,mp-1185702,-0.031714,0.0000,Mg16Al12Ru,0.070023,"['Al', 'Mg', 'Ru']",# generated using pymatgen\ndata_Mg16Al12Ru\n_...,160


In [504]:
cifs = {}
for df in ms:
    ff = df[['material_id', 'cif']].to_dict()
    cifs = cifs | {ff['material_id'][i]:ff['cif'][i] for i in range(len(ff['cif']))}

In [509]:
cnt = []
for id in tqdm(cifs):
    if id + '.cif' in set(mids):
        continue
    else:
        cnt.append(id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140374/140374 [16:28<00:00, 142.04it/s]


In [518]:
for i in tqdm(cnt):
    with open(out_mp + '/' + i + '.cif', 'w') as f:
        f.write(cifs[i])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8381/8381 [00:06<00:00, 1254.27it/s]


# AMSCD

In [178]:
import os
from tqdm import tqdm

ruff_path = "/scratch/civil/phd/cez198233/vaibhav_nlp/robocif/amcsd"
ruff_files = os.listdir(ruff_path)
ruff_cifs = [os.listdir(os.path.join(ruff_path, i)) for i in ruff_files]

In [185]:
ruff_cifs_path = []
for _, pth in enumerate(ruff_cifs):
    for _pth in pth:
        if _pth.endswith('.cif'):
            ruff_cifs_path.append(os.path.join(ruff_path, ruff_files[_], _pth))